In [2]:
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

import json
import pprint
from pymongo import MongoClient

In [2]:
year = ['2016', '2017', '2018', '2019']

df_list = {}
for y in year:
    file_name = f"Resources/san_francisco-censustracts-{y}-3-All-HourlyAggregate.csv"
    df_list[y] = pd.read_csv(file_name, usecols=['sourceid', 'dstid', 'hod', 'mean_travel_time', 'standard_deviation_travel_time'])

In [3]:
for y in year:
    print(df_list[y].head())
    print('------------------------------')

   sourceid  dstid  hod  mean_travel_time  standard_deviation_travel_time
0         7      6   23            258.13                          186.05
1         9      2   23           2241.63                          308.47
2         3     17   15           3403.58                          544.78
3         3     38    5            305.63                          152.80
4         2     61   11           1324.60                          237.68
------------------------------
   sourceid  dstid  hod  mean_travel_time  standard_deviation_travel_time
0         6      5    1           2324.14                          590.86
1         6      7    7            748.86                          286.52
2         2     20   14           3377.44                          420.10
3         3     17    1           3035.29                          163.68
4         3     38   20            334.59                          260.88
------------------------------
   sourceid  dstid  hod  mean_travel_time  standar

In [4]:
engine = create_engine("sqlite:///Resources/files.sqlite")
conn = engine.connect()

In [5]:
Base = automap_base()

In [6]:
Base.prepare(engine, reflect=True)

In [7]:
for y in year:
    df_list[y].to_sql(y,conn, if_exists='replace', index = False)

In [8]:
inspector = inspect(engine)
inspector.get_table_names()

['2016', '2017', '2018', '2019']

In [9]:
path = os.path.join('Resources', 'san_francisco_censustracts.json')
with open(path) as json_file:
    data = json.load(json_file)

In [10]:
client = MongoClient('localhost', 27017)
db = client['map_tile']
db.bay_area.drop()
bay_area = db['bay_area']
bay_area.insert_one(data)

In [11]:
# example=bay_area.find()
# pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(example)
# print(type(example))
# for i in example:
#     print(i)